In [5]:
import pulp
import pandas as pd

In [6]:
from typing import Dict

POINTS_TO_CASH: Dict[str, Dict[str, float]]= {
    "default": {
        "miles": 0.01,
        "points": 0.01
    },

    "American Express": {
        "miles": 0.016,
        "points": 0.006
    },

    "Capital One": {
        "miles": 0.016,
        "points": 0.005
    },

    "Chase": {
        "miles": 0.018,
        "points": 0.008
    },

    "Citi": {
        "miles": 0.016,
        "points": 0.01,
    },

    "Bank of America": {
        "miles": 0.01,
        "points": 0.006
    },

    "Wells Fargo": {
        "miles": 0.01,
        "points": 0.01
    },

    "U.S. Bank": {
        "miles": 0.015,
        "points": 0.012,
    },

    "Discover": {
        "miles": 0.01,
        "points":  0.01
    }
}


In [7]:
def get_conversion_rate(issuer: str, unit: str) -> float:
    # pick issuer or default
    issuer_table = POINTS_TO_CASH.get(issuer, POINTS_TO_CASH["default"])

    # try issuer-specific valuation
    if unit in issuer_table:
        return issuer_table[unit]

    # fallback to default valuation for this unit
    return POINTS_TO_CASH["default"].get(unit, 0.01)


In [8]:
def get_dicts(df):
    cards_dict = {}
    fees_dict = {}

    for _, row in df.iterrows():
        card_name = row["name"]
        issuer = row["issuer"]        # ADDED FOR ANGELA FRAMEWORK
        fees_dict[card_name] = row["clean_annual_fee"]

        rates = {}
        triggers = {}

        for reward in row["clean_rewards"]:
            if not reward:
                continue

            value = reward[0]
            rtype = reward[1]
            unit = reward[2]             # "miles", "points", etc.
            category = reward[3]         # direct category from  dataset

            # issuer based conversion
            conversion = get_conversion_rate(issuer, unit)
            cash_rate = value * conversion

            if rtype in ("Multiplier", "Per Unit"):
                # keep best rate for this category
                rates[category] = max(rates.get(category, 0), cash_rate)

            elif rtype == "Trigger":
                # placeholder for future
                triggers.setdefault(category, []).append((row["min_spend"], cash_rate))

        cards_dict[card_name] = {
            "min_score": 0,
            "rates": rates,
            "triggers": triggers,
        }

    return cards_dict, fees_dict


In [9]:
def summarize(cats, chosen, spending, cards, trigger_bonus, fees, held):
    breakdown = {}

    for k in cats:
        c = chosen[k]                      # card chosen for category k
        spend = spending[k]                # dollars spent in that category
        rate = cards[c]["rates"].get(k, 0.0)
        trig = trigger_bonus[(c, k)]       # from your precomputed trigger table
        reward_val = spend * rate + trig
        fee_val = fees[c] if c in held else 0   # fee counted once if held
        breakdown[k] = {
            "card": c,
            "spend": spend,
            "rate": rate,
            "trigger_bonus": trig,
            "raw_reward": spend * rate,
            "total_reward": reward_val,
            "fee": fee_val,
            "net_contribution": reward_val - fee_val,
            "equation": f"{spend} * {rate} + {trig} - {fee_val}"
        }
    return breakdown

In [10]:
def trigger_bonuses(card_list, cards, spending, cats):
    bonuses={}
    for c in card_list:
        trig_dict = cards[c].get("triggers", {})  # may be missing
        for k in cats:
            total_bonus = 0.0
            if k in trig_dict:
                s = spending[k]
                for min_spend, bonus in trig_dict[k]:
                    if s >= min_spend:
                        total_bonus += bonus
            bonuses[(c, k)] = total_bonus
    return bonuses

In [11]:
def optimize_cardspace(cards, fees, spending, score):
    """
    cards[card] = {"min_score": int, "rates": {category: rate}}
    fees[card]  = annual fee
    spending[category] = spend in that category
    """
    # For Trigger-Based: 
    # Given spending amount in a category you can determine if they will hit a trigger and just add that to reward
    # Note: its hard to account for triggers that might not stack. for ex if you spend 100 get 20 but if you spend 200 and get 40 you shouldnt apply both.
    # Im gonna assume that if you exceed the spend for a trigger that ur gonna get that bonus
    
    # For point distribution: TODO
    # Take point equivalents by category consider combinations with constraint as points earned. Create card copies

    prob = pulp.LpProblem("Maximize_Rewards", pulp.LpMaximize)
    cats = list(spending.keys())
    card_list = list(cards.keys())

    #eligibility filter from score
    eligible = {c: 1 if score >= cards[c].get("min_score", 0) else 0 for c in card_list}

    #decision vars

    #card selection indicator
    y = {c: pulp.LpVariable(f"hold_{c}", 0, 1, pulp.LpBinary) for c in card_list}

    #category decision 
    x = {(c,k): pulp.LpVariable(f"use_{c}_{k}", 0, 1, pulp.LpBinary)
         for c in card_list for k in cats}

    # Precompute trigger bonuses per (card, category)
    # If spending in k exceeds min_spend for a trigger, include its bonus.
    trigger_bonus = trigger_bonuses(card_list, cards, spending, cats)


    #objective: rewards - fees
    reward = pulp.lpSum((spending[k] * cards[c]["rates"].get(k, 0.0) + trigger_bonus[(c, k)]) * x[(c,k)] 
                        for c in card_list for k in cats)
    fee = pulp.lpSum(fees[c] * y[c] for c in card_list)
    prob += reward - fee

    #at most one card per category
    for k in cats:
        prob += pulp.lpSum(x[(c,k)] for c in card_list) <= 1

    #can only use a card if you hold it
    for c in card_list:
        for k in cats:
            prob += x[(c,k)] <= y[c]
            

    for c in card_list:
        #can only hold if eligible by score
        prob += y[c] <= eligible[c]
        # If a card is held, it must be used in at least one category. (Avoids treating 0 fee cards as free)
        prob += pulp.lpSum(x[(c,k)] for k in cats) >= y[c]


    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    if pulp.LpStatus[pulp.LpStatusOptimal] != 'Optimal' and pulp.LpStatus[prob.status] != 'Optimal':
        return "No optimal solution found", 0.0, set(), {}

    total = pulp.value(reward - fee)
    chosen = {k: max(card_list, key=lambda c: pulp.value(x[(c,k)])) for k in cats}
    held   = {c for c in card_list if pulp.value(y[c]) > 0.5}
    
    breakdown = summarize(cats, chosen, spending, cards, trigger_bonus, fees, held)
    return chosen, total, held, breakdown


In [12]:
#Test case: normal
cards1 = {
    "A": {"min_score": 680, "rates": {"groceries": 0.03, "travel": 0.01}},
    "B": {"min_score": 700, "rates": {"groceries": 0.01, "travel": 0.02}},
}
fees1 = {"A": 95, "B": 95}
spending1 = {"groceries": 5000, "travel": 4000}
score1 = 720

#should just hold A bc pnl for B is 80-95=-15.

#Test case: zero spend
cards2 = {
    "A": {"min_score": 680, "rates": {"groceries": 0.04, "dining": 0.01}},
    "B": {"min_score": 650, "rates": {"groceries": 0.015, "dining": 0.015}},
}
fees2 = {"A": 95, "B": 0}
spending2 = {"groceries": 3000, "dining": 0}
score2 = 700

#should only hold B bc pnl for A is negative

# Test case: High fee dominates
cards3 = {
    "A": {"min_score": 740, "rates": {"travel": 0.05}},
    "B": {"min_score": 680, "rates": {"travel": 0.02}},
}
fees3 = {"A": 50, "B": 0}
spending3 = {"travel": 2000}
score3 = 800

#Gain 40 for holding B, gain 50 for holding A. Should hold both if B is free. 

# Test case: Inelligible by score
cards4 = {
    "A": {"min_score": 760, "rates": {"gas": 0.04}},
    "B": {"min_score": 600, "rates": {"gas": 0.01}},
}
fees4 = {"A": 0, "B": 0}
spending4 = {"gas": 1000}
score4 = 500

#Test case: Tie
cards5 = {
    "A": {"min_score": 650, "rates": {"dining": 0.03}},
    "B": {"min_score": 650, "rates": {"dining": 0.03}},
}
fees5 = {"A": 0, "B": 0}
spending5 = {"dining": 2500}
score5 = 700

#Test case: Strictly Negative
cards6 = {
    "A": {"min_score": 650, "rates": {"dining": 0.03}},
    "B": {"min_score": 650, "rates": {"dining": 0.03}},
}
fees6 = {"A": 10000, "B": 10000}
spending6 = {"dining": 2500}
score6 = 700

In [13]:
def run_case(n, cards, fees, spending, score):
    print(f"\nCase {n}")
    chosen, total, held, breakdown = optimize_cardspace(cards, fees, spending, score)
    print("Held:", held)
    print("Assignments:", chosen)
    print("Net reward ($):", round(total, 2))
    print("Breakdown:", breakdown)

for i, data in enumerate([
    (cards1, fees1, spending1, score1),
    (cards2, fees2, spending2, score2),
    (cards3, fees3, spending3, score3),
    (cards4, fees4, spending4, score4),
    (cards5, fees5, spending5, score5),
    (cards6, fees6, spending6, score6)
], start=1):
    run_case(i, *data)


Case 1
Held: {'A'}
Assignments: {'groceries': 'A', 'travel': 'A'}
Net reward ($): 95.0
Breakdown: {'groceries': {'card': 'A', 'spend': 5000, 'rate': 0.03, 'trigger_bonus': 0.0, 'raw_reward': 150.0, 'total_reward': 150.0, 'fee': 95, 'net_contribution': 55.0, 'equation': '5000 * 0.03 + 0.0 - 95'}, 'travel': {'card': 'A', 'spend': 4000, 'rate': 0.01, 'trigger_bonus': 0.0, 'raw_reward': 40.0, 'total_reward': 40.0, 'fee': 95, 'net_contribution': -55.0, 'equation': '4000 * 0.01 + 0.0 - 95'}}

Case 2
Held: {'B'}
Assignments: {'groceries': 'B', 'dining': 'A'}
Net reward ($): 45.0
Breakdown: {'groceries': {'card': 'B', 'spend': 3000, 'rate': 0.015, 'trigger_bonus': 0.0, 'raw_reward': 45.0, 'total_reward': 45.0, 'fee': 0, 'net_contribution': 45.0, 'equation': '3000 * 0.015 + 0.0 - 0'}, 'dining': {'card': 'A', 'spend': 0, 'rate': 0.01, 'trigger_bonus': 0.0, 'raw_reward': 0.0, 'total_reward': 0.0, 'fee': 0, 'net_contribution': 0.0, 'equation': '0 * 0.01 + 0.0 - 0'}}

Case 3
Held: {'A'}
Assignment

In [14]:
df= pd.read_json("clean_cards.json")

df.head()


,name,issuer,annual_fee,rewards,clean_annual_fee,clean_rewards
0,Platinum Mastercard,Capital One,$0,[Get the credit you need with no annual fee.],0,[]
1,Venture X Rewards,Capital One,$395,[Earn unlimited 10X miles on hotels and rental...,395,"[[10.0, Multiplier, miles, Travel], [5.0, Mult..."
2,Venture Rewards,Capital One,$95,[Earn unlimited 2 miles per dollar on every pu...,95,"[[2.0, Per Unit, miles, All Purchases], [5.0, ..."
3,VentureOne Rewards,Capital One,$0,[Earn unlimited 1.25 miles per dollar on every...,0,"[[1.25, Per Unit, miles, All Purchases], [5.0,..."
4,VentureOne Rewards for Good Credit,Capital One,$0,[Earn unlimited 1.25 miles per dollar on every...,0,"[[1.25, Per Unit, miles, All Purchases]]"


In [15]:
cards, fees= get_dicts(df)


In [18]:
spending = {"Groceries & Dining": 5000, "Travel": 4000, "All Purchases": 10000}
chosen, total, held, breakdown = optimize_cardspace(cards, fees, spending, 800)
total

1173.0